# [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic) #

Objetivo é desenvolver um modelo preditivo que responda a questão: que tipos de pessoas tinham mais chance de sobreviver? Usando para isso os dados como nome, idade, gênero, classe sócio econômica, etc.

Resolução baseada no tutorial do [Mario Filho](https://github.com/ledmaster/TutorialTitanic)

## Bibliotecas ##

In [167]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Dados ##

Leitura dos dados

In [168]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv("data/test.csv")

# data_train.head()

Quantizando os dados

In [169]:
def quant_sex(value):
    ''' Change variable Sex from qualitative to quantitative values'''
    if value == 'female':
        return 1
    else:
        return 0

## Modelos ##

Selecionando variavéis de interesse

In [170]:
variables = ['Sex_bin', 'Age']

data_train['Sex_bin'] = data_train['Sex'].map(quant_sex)
sub_data_train = data_train[variables]
sub_data_train = sub_data_train.fillna(-1)
survived_train = data_train['Survived']

data_test['Sex_bin'] = data_test['Sex'].map(quant_sex)
sub_data_test = data_test[variables]
sub_data_test = sub_data_test.fillna(-1)

### Base ###

Modelo que assume que todas as mulheres sobrevivem

In [177]:
baseline = (data_test['Sex'] == 'female').astype(int)
baseline.index = data_test['PassengerId']
baseline.name = 'Survived'
baseline.to_csv('data/gender_submission.csv', header = True)

# Imprime os 10 primeiros registros do arquivo
# !head -n10 data/gender_submission.csv

### Random Forest ###

Criando modelo, executando treinamento e predição

In [97]:
model = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 0)

model.fit(sub_data_train, survived_train) 

survived_predicted_test = model.predict(sub_data_test)

# Imprime variavel
# survived_predicted_test

Exportando resultados para formato de submissão

In [126]:
output = pd.Series(survived_predicted_test, index = data_test['PassengerId'], name = 'Survived')
output.to_csv("data/random_forest_SA.csv", header = True)

# !head -n10 random_forest_SA.csv

### Regressão Linear ###

## Validação ##

In [204]:
sub_train, sub_valid, sub_surv_train, sub_surv_valid  = train_test_split(sub_data_train, survived_train, test_size = 0.5)

# Imprime dimensões das tuplas
# sub_train.shape, sub_valid.shape, sub_surv_train.shape, sub_surv_valid.shape

# Baseline
baseline = (sub_valid['Sex_bin'] == 1).astype(np.int64)

# Random Forest
model = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 0)
model.fit(sub_train, sub_surv_train) 
survived_predicted_valid_RF = model.predict(sub_valid)

print('Baseline\t', np.mean(sub_surv_valid == baseline), '\nRandom Forest\t', np.mean(sub_surv_valid == survived_predicted_valid_RF))

Baseline	 0.7802690582959642 
Random Forest	 0.7645739910313901
